In [ ]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

In [ ]:
#Script to write csv to use in Deep Learning 

In [ ]:
json_path = '/home/deeplearning/AHL/AHL_new_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

In [ ]:
study_uid = []

for data in dataset : 
    study_uid.append(data[4])

In [ ]:
#get patient_name from json 
liste_json = []
list_dir = os.listdir('/home/deeplearning/AHL/AHL_JSON')
list_dir.remove('AHL_JSON_1')
for file_ in list_dir : 
    liste_json.append(os.path.join('/home/deeplearning/AHL/AHL_JSON', file_))



for data in dataset : 
    for json_path in liste_json : 
        with open(json_path) as json_file : 
            reader = json.load(json_file)

            uid = reader['study']['StudyInstanceUID']

            if uid == data[4] : 

                data.append(reader['patient']['PatientName'])

                break


In [ ]:
#liste of generated nifti by "nifti_builder_from_json" script 

nifti_directory = '/home/deeplearning/AHL/AHL_NIFTI'
liste_n = os.listdir(nifti_directory)

liste_nifti_pet0 = []
for l in liste_n : 
    liste_nifti_pet0.append(os.path.join('/home/deeplearning/AHL/AHL_NIFTI', l))

pet_2 = os.listdir('/home/deeplearning/AHL/AHL_NIFTI/pet2')
liste_nifti_pet2 = []
for pet in pet_2 : 
    liste_nifti_pet2.append(os.path.join('/home/deeplearning/AHL/AHL_NIFTI/pet2', pet))

pet_4 = os.listdir('/home/deeplearning/AHL/AHL_NIFTI/pet4')
liste_nifti_pet4 = []
for pet in pet_4 : 
    liste_nifti_pet4.append(os.path.join('/home/deeplearning/AHL/AHL_NIFTI/pet4', pet))


liste_nifti = liste_nifti_pet0 + liste_nifti_pet2 + liste_nifti_pet4

no_nifti = []
for nifti in liste_nifti : 
    if "nifti" not in nifti : 
        no_nifti.append(nifti)

for fold in no_nifti : 
    liste_nifti.remove(fold)

In [ ]:
len(liste_nifti)

In [ ]:
for row in dataset : 
    subsubliste = []
    for nifti in liste_nifti : 
        if row[4] in nifti: 
            subsubliste.append(nifti)
    if 'PET0' in row : 
        if len(subsubliste) == 3 : #or == 3 if there is a mask 
            subsubliste = sorted(subsubliste)
            row.append(subsubliste[0])
            row.append(subsubliste[1])
            row.append(subsubliste[2]) #add it if there is a mask 
        else : 
            pass
    elif 'PET2' in row or 'PET4' in row : 
        if len(subsubliste) == 2  : #or == 3 if there is a mask 
            subsubliste = sorted(subsubliste)
            row.append(subsubliste[0])
            row.append(subsubliste[1])

    #subliste.append(type_)
    #liste.append(subliste)

In [ ]:
#clean serie : must have patient_name, patient ID, study_UID, CT_path, PET_path, MASK_path if pet0, and type "PET" (0,2,4)
completed_liste = []
for serie in dataset : 
    if len(serie) == 10 or len(serie) == 11 : #== 10 if Mask_path 
        completed_liste.append(serie)

In [ ]:
len(completed_liste)

In [ ]:
#here check for double : if there are 2 patient with same ID, or same STUDY_UID, delete the other(s) serie(s)

dataset = []
dataset.append(completed_liste[0])
for i in range(1, len(completed_liste)) :
    if completed_liste[i] not in dataset : 
        dataset.append(completed_liste[i])

In [ ]:
print("nombre de study : ", len(dataset))

In [ ]:
#create folder to save nifti

for serie in dataset: 
    if 'PET0' in serie : 
        patient_name = serie[7]
        patient_id = serie[6]
        pet_type = serie[5]
        nifti_ct = serie[8]
        nifti_pet = serie[9]
        nifti_mask = serie[10] #if pet0

        folder = '/home/deeplearning/AHL/AHL_NIFTI'+'/'+patient_id

        if not os.path.exists(folder):
            os.makedirs(folder)
            print(folder, 'created')
        else : print(folder, "Dossier patient déjà crée")

        subfolder = folder+'/'+pet_type
        if not os.path.exists(subfolder) :
            os.makedirs(subfolder)
            print(subfolder, 'created')
        else : print(subfolder, "Déjà creer => already folder for ", pet_type)

        ct_path = nifti_ct
        pt_path = nifti_pet
        mask_path = nifti_mask

        new_ct_path = subfolder+'/'+nifti_ct.split('/')[-1]
        new_pet_path = subfolder+'/'+nifti_pet.split('/')[-1]
        new_mask_path = subfolder+'/'+nifti_mask.split('/')[-1]
        try : 

            os.rename(ct_path, new_ct_path)
            os.rename(pt_path, new_pet_path)
            os.rename(mask_path, new_mask_path)

            serie[8] = new_ct_path
            serie[9] = new_pet_path 
            serie[10] = new_mask_path
        except Exception as err : 
            print(err)
    else : 
        patient_name = serie[7]
        patient_id = serie[6]
        pet_type = serie[5]
        nifti_ct = serie[8]
        nifti_pet = serie[9]

        folder = '/home/deeplearning/AHL/AHL_NIFTI'+'/'+patient_id

        if not os.path.exists(folder):
            os.makedirs(folder)
            print(folder, 'created')
        else : print(folder, "Dossier patient déjà crée")

        subfolder = folder+'/'+pet_type
        if not os.path.exists(subfolder) :
            os.makedirs(subfolder)
            print(subfolder, 'created')
        else : print(subfolder, "Déjà creer => already folder for ", pet_type)

        ct_path = nifti_ct
        pt_path = nifti_pet

        new_ct_path = subfolder+'/'+nifti_ct.split('/')[-1]
        new_pet_path = subfolder+'/'+nifti_pet.split('/')[-1]

        try : 
            os.rename(ct_path, new_ct_path)
            os.rename(pt_path, new_pet_path)

            serie[8] = new_ct_path
            serie[9] = new_pet_path 

        except Exception as err : 
            print(err)

In [ ]:
#write csv , add NIFTI_MASK column if there is a mask 
nifti_directory = '/home/deeplearning/AHL'
filename = 'AHL_NIFTI_V2.csv'


with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT NAME", "PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK"])
    for serie in dataset : 
        if 'PET0' in serie : 
            csv_writer.writerow([serie[7], serie[6], serie[4], serie[5].lower(), serie[8], serie[9], serie[10]])
        else : 

            csv_writer.writerow([serie[7], serie[6], serie[4], serie[5].lower(), serie[8], serie[9], ""])